### Imports

In [77]:
import sys
import os
import os.path
print(os.getcwd())
#sys.path.append('C:\PROGRA~1\anaconda3\Lib\site-packages')
sys.executable

C:\Users\bsmartinez\Documents\PROYECTOS\NER\NER_\Python


'C:\\ProgramData\\Anaconda3\\python.exe'

In [78]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "..\\NER Translator-510221cde3c4.json"
print(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))

..\NER Translator-510221cde3c4.json


In [79]:
import time

In [80]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('ca')

In [81]:
import nltk
from nltk.tag.stanford import StanfordNERTagger

In [82]:
from google.cloud import translate
translator = translate.Client()

In [83]:
import json

In [84]:
import ahocorasick
A = ahocorasick.Automaton()

In [85]:
import re

# Settings

In [86]:
rehacer_textos_completados = True   # (rehacer supone que ya estan traducidos y que es rehacer el resto)

# Files

In [87]:
Stanford_NER_jar = '../Java/stanford-ner-2018-10-16/stanford-ner.jar'
# ingles
model_en = '../Java/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz'
# castellano
model_es = '../Java/stanford-ner-2018-10-16/classifiers/spanish.ancora.distsim.s512.crf.ser.gz'

In [88]:
ner_tagger_en = StanfordNERTagger(model_en, Stanford_NER_jar, encoding='utf8')
ner_tagger_es = StanfordNERTagger(model_es, Stanford_NER_jar, encoding='utf8')

In [89]:
ruta = r"..\\Files\\"
ruta_textos = r"..\\Files\\textos\\"
ruta_tokenizados = r"..\\Files\\tokenizados\\"
ruta_etiquetados = r"..\\Files\\etiquetados\\"
ruta_traducidos = r"..\\Files\\traducidos\\"

In [90]:
f_info = open(ruta + "Info.noTocar", "r", encoding ='utf-8')
info_0 = f_info.read()
f_info.close()

In [91]:
print("# Textos ya completados: \n" + info_0)
info = info_0.split("\n")
for idx, key in enumerate(info):
    A.add_word(key, (idx, key))

# Textos ya completados: 
﻿Regne Unit
Segona Guerra Mundial
Europa
Unió Europea
Joan d'Aragó (príncep d'Astúries)
SEAT
Catalana Occident
Vueling Airlines
Aeroport Josep Tarradellas Barcelona-El Prat
Barcelona
Ajuntament de Barcelona
Fuita de seus socials de Catalunya de 2017
Joaquim Torra i Pla
Banc Sabadell
Economia de Catalunya
La Caixa
Grifols
Spanair
Universitat Pompeu Fabra
Fira de Barcelona
José María Aznar López
Citroën
Eusebi Millan i Alonso
Lionel Andrés Messi
Òmnium Cultural
Josep Guardiola i Sala
Sant Cugat del Vallès
Hispano-Suiza
Forces Elèctriques de Catalunya
Roca (empresa)
NASA
El Bulli
Premi Nobel
World Wide Web



# Cargar páginas de wikipedia para entrenar

In [92]:
lista_paginas = ["Albert Einstein"]
#
n_paginas = len(lista_paginas)

In [93]:
indices_textos_a_modificar = []

print("#Lista Páginas: \n")

textos_splitted = []
for i in range(0,n_paginas):
    if lista_paginas[i]+ ".txt" not in os.listdir(ruta_textos):
        textos_splitted.append([])
        with open(ruta_textos + lista_paginas[i] + ".txt", "w", encoding ='utf-8') as f:
            texto = wiki_wiki.page(lista_paginas[i]).text
            texto_splt = []
            pos = 0
            while pos > -1:
                pos = re.search(r'([0-9]{2}|[a-z]{2})\.[ \n]',texto)
                if pos != None:
                     pos = pos.span()[0]
                else:
                    pos = -1
                    continue
                texto_splt.append(texto[:pos + 2])
                texto = texto[pos + 4:]
            for sentence in texto_splt:
                sentence_trimmed = sentence.replace('\n', '').strip()
                if len(sentence_trimmed) > 1: 
                    f.write(sentence_trimmed + "\n")
                    textos_splitted[i].append(sentence_trimmed)
            print("New. ---  " + lista_paginas[i])
            f.close()
    else:
        if rehacer_textos_completados == False:
            n_paginas = n_paginas - 1
            if n_paginas > 0:
                lista_paginas.remove(i)
            else:
                lista_paginas = []
        else:
            indices_textos_a_modificar.append(i)
            with open(ruta_textos + lista_paginas[i] + ".txt", "r", encoding ='utf-8') as f:
                textos_splitted.append(re.split(r"\n",f.read()))
            f.close()
            print("Mod. ---  " + lista_paginas[i])

#Lista Páginas: 

Mod. ---  Albert Einstein


# Llamar al .jar que tokeniza los textos en Catalán (hacerlo a mano !!)

In [94]:
# si esto da False, ejecutar a mano:
#0 == os.system("java -jar Tokenizador_Cat.jar")
#0 == os.system("java -jar ..\\Java\\Tokenizador_Cat\\Tokenizador_Cat.jar")

# Cargar Textos Tokenizados

In [95]:
def left(s, amount):
    return s[:amount]

In [96]:
textos_tokenizados = []
for i in range(0,n_paginas):
    with open(ruta_tokenizados + lista_paginas[i] + ".txt", "rb") as f:
        textos_tokenizados.append(f.read().decode('utf-8').rstrip()) #textos_tokenizados

# Matriz Items

In [97]:
matriz_longitudes = []

In [98]:
textos_items_splitted = []
for i in range(0,n_paginas):
    print("\n" + lista_paginas[i].upper() + " (" + str(i+1) + " de " + str(n_paginas) + ")")
    paragrafos = textos_tokenizados[i].split("\n\n")  
    n_paragrafos = len(paragrafos)
    textos_items_splitted.append([])
    matriz_longitudes.append([])
    for j in range(0,n_paragrafos):
        tokens = paragrafos[j].split("\n")
        n_tokens = len(tokens)
        matriz_longitudes[i]
        print("Sentence " + str(j+1) + ": " + str(n_tokens) + " items")
        textos_items_splitted[i].append([])
        matriz_longitudes[i].append([])
        for token in tokens:
            textos_items_splitted[i][j].append((token,'O'))
            matriz_longitudes[i][j] = n_tokens


ALBERT EINSTEIN (1 de 1)
Sentence 1: 97 items
Sentence 2: 28 items
Sentence 3: 33 items
Sentence 4: 22 items
Sentence 5: 23 items
Sentence 6: 20 items
Sentence 7: 30 items
Sentence 8: 62 items
Sentence 9: 17 items
Sentence 10: 10 items
Sentence 11: 15 items
Sentence 12: 19 items
Sentence 13: 18 items
Sentence 14: 6 items
Sentence 15: 16 items
Sentence 16: 15 items
Sentence 17: 67 items
Sentence 18: 65 items
Sentence 19: 14 items
Sentence 20: 11 items
Sentence 21: 62 items
Sentence 22: 32 items
Sentence 23: 17 items
Sentence 24: 29 items
Sentence 25: 28 items
Sentence 26: 78 items
Sentence 27: 24 items
Sentence 28: 93 items
Sentence 29: 32 items
Sentence 30: 19 items
Sentence 31: 56 items
Sentence 32: 16 items
Sentence 33: 8 items
Sentence 34: 19 items
Sentence 35: 20 items
Sentence 36: 17 items
Sentence 37: 20 items
Sentence 38: 19 items
Sentence 39: 94 items
Sentence 40: 19 items
Sentence 41: 16 items
Sentence 42: 28 items
Sentence 43: 33 items
Sentence 44: 26 items
Sentence 45: 58 i

# Gazettes 


## Cargarlas

In [99]:
gazettes = []
gazettes_names = []

In [100]:
for gazette_file in os.listdir("..\\Files\\gazettes"):
    gazettes_names.append(left(gazette_file, len(gazette_file)-4))
    with open("..\\Files\\gazettes\\" + gazette_file, "rb") as f:
        gazettes.append(f.read().decode('latin-1').split("\r\n"))

## Aplicarlas

In [101]:
for i in range(0,len(matriz_longitudes)):
    for j in range(0,len(matriz_longitudes[i])):
        l = matriz_longitudes[i][j]
        n_gz = len(gazettes)
        for gazette_index in range(0,n_gz):
            for i_gazette in gazettes[gazette_index]:
                l_i_gazette = len(i_gazette.split(" "))
                for k in range(0,l):
                    if textos_items_splitted[i][j][k][1] == 'O':
                        item = ""
                        for a in range(0, l_i_gazette):                         
                            if k + a < l:
                                if a == 0:
                                    item = item + textos_items_splitted[i][j][k + a][0]
                                else:
                                    item = item + " " + textos_items_splitted[i][j][k + a][0]
                        #print(item)
                        if i_gazette.lower() == item.lower():
                            for a in range(0, l_i_gazette):    
                                new_item = (textos_items_splitted[i][j][k + a][0],gazettes_names[gazette_index])
                                textos_items_splitted[i][j][k + a] = new_item   

## Aplicarlas Aho-Corasick

n_gz = len(gazettes)
for p in range(0,n_paragrafos):
    for idx, key in enumerate(texto_items_splitted[p]):
        A.add_word(key, (idx, key))
    A.make_automaton()  
    for gazette_index in range(0,n_gz):
        for i_gazette in gazettes[gazette_index]:
    
    #A.clear() ??

# Traducciones

## castellano

## inglés

In [102]:
textos_en_splitted = []
excepcion_yet = False
indices_textos_nuevos_sin_traduccion = []

for i in range(0,len(matriz_longitudes)):
    file_name = ruta_traducidos + lista_paginas[i] + ".noTocar"
    if os.path.isfile(file_name):
        with open(file_name, "r", encoding ='utf-8') as f:
            textos_en_splitted.append(re.split(r'\n',f.read()))   
            f.close()
    else:
        with open(file_name, "w", encoding ='utf-8') as f:
            textos_en_splitted.append([])
            for j in range(0,len(matriz_longitudes[i])):
                #textos_en_splitted[i].append([])
                try:
                    #texto_en = translator.translate(textos_splitted[i][j], dest= 'en').text
                    json_trans = translator.translate(textos_splitted[i][j], target_language = 'en')    
                    texto_en = json.loads(json.dumps(json_trans))['translatedText']
                    time.sleep(0.2)
                    textos_en_splitted[i].append(texto_en)
                    f.write(texto_en + "\n")
                except:
                    if not excepcion_yet:
                        print("No se han terminado de traducir: \n")
                    excepcion = True
                    print(lista_paginas[i])
                    indices_textos_nuevos_sin_traduccion.append(i)
                        #textos_en_splitted.remove(i)
                    break
            f.close()

## intentar seguir traduciendo (se puede ejecutar las veces que se quiera)

In [103]:
excepcion = False
indices_textos_nuevos_sin_traduccion_otra_vez = []

for i in indices_textos_nuevos_sin_traduccion:
    file_name = ruta_traducidos + lista_paginas[i] + ".noTocar"
    with open(file_name, "a", encoding ='utf-8') as f:
        for j in range(len(textos_en_splitted[i]),len(matriz_longitudes[i])):
            try:
                #texto_en = translator.translate(textos_splitted[i][j], dest= 'en').text
                json_trans = translator.translate(textos_splitted[i][j], target_language = 'en')
                texto_en = json.loads(json.dumps(json_trans))['translatedText']
                time.sleep(0.1)
                textos_en_splitted[i].append(texto_en)
                f.write(texto_en + "\n")
            except:
                if not excepcion:
                    print("Aún no se han terminado de traducir: \n")
                excepcion = True
                print(lista_paginas[i])
                indices_textos_nuevos_sin_traduccion_otra_vez.append(i)
                    #textos_en_splitted.remove(i)
                break
        f.close()
        
indices_textos_nuevos_sin_traduccion = indices_textos_nuevos_sin_traduccion_otra_vez

## seguir con lo que hay

In [104]:
lista_paginas = [i for j, i in enumerate(lista_paginas) if j not in indices_textos_nuevos_sin_traduccion]
n_paginas = n_paginas - len(indices_textos_nuevos_sin_traduccion)
matriz_longitudes = [i for j, i in enumerate(matriz_longitudes) if j not in indices_textos_nuevos_sin_traduccion]
textos_items_splitted = [i for j, i in enumerate(textos_items_splitted) if j not in indices_textos_nuevos_sin_traduccion]
textos_en_splitted = [i for j, i in enumerate(textos_en_splitted) if j not in indices_textos_nuevos_sin_traduccion]

## Tokenizar Traducciones 

In [105]:
textos_words_en_splitted = []
for i in range(0,len(matriz_longitudes)):
    textos_words_en_splitted.append([])
    for j in range(0,len(matriz_longitudes[i])):
        textos_words_en_splitted[i].append(nltk.word_tokenize(textos_en_splitted[i][j]))

## NER Tagger Traducciones

In [106]:
textos_items_en_splitted = []
for i in range(0,len(textos_words_en_splitted)):
    textos_items_en_splitted.append([])
    for j in range(0,len(textos_words_en_splitted[i])):
        textos_items_en_splitted[i].append(ner_tagger_en.tag(textos_words_en_splitted[i][j]))

## Visualización Tags Traducciones

In [107]:
def tag_to_background_color_en(argument):
    switcher = {
        'PERSON' : 'orange',
        'ORGANIZATION' : 'red',
        'LOCATION' : 'magenta',
    }
    return switcher.get(argument, "white")

In [108]:
from IPython.core.display import display, HTML
html = ""
for paragrafo in textos_items_en_splitted[0]:
    html = html + '<p'
    for item in paragrafo:
            html = html + '<span style = "background-color: ' + tag_to_background_color_en(item[1]) + ';">' + item[0] + ' </span>'
    html = html + '</p>'
display(HTML(html))

# Taggear Catalan a partir de los tags de las Traducciones

In [109]:
for i in range(0,len(matriz_longitudes)):
    for j in range(0,len(matriz_longitudes[i])):
        l = matriz_longitudes[i][j]
        l_en = len(textos_items_en_splitted[i][j])
        for k_en in range(0, l_en):
            item_en = textos_items_en_splitted[i][j][k_en]
            for k in range(0,l):   
                item = textos_items_splitted[i][j][k]
                if item_en[1] != 'O':
                    if item[1] == 'O':
                        # cuando puede que el traductor haya traducido al ingles
                        if item_en[1] == 'LOCATION':
                            json_trans = translator.translate(item_en[0],source_language='en', target_language = 'ca')                  
                            texto_en = json.loads(json.dumps(json_trans))['translatedText']
                            item_en = (texto_en,'LOCATION')
                            #time.sleep(0.5)
                        elif item_en[1] == 'ORGANIZATION':
                            json_trans = translator.translate(item_en[0],source_language='en', target_language = 'ca')                  
                            texto_en = json.loads(json.dumps(json_trans))['translatedText']
                            item_en = (texto_en,'ORGANIZATION')
                        # cuando normalmente el traductor deja el original
                        if item_en[0]==item[0]:
                            if k_en < l_en-1 and 0 < k < l-1:
                                if textos_items_en_splitted[i][j][k_en + 1][1] != 'O':
                                    traduccion = json.loads(json.dumps(translator.translate(textos_items_en_splitted[i][j][k_en + 1][0],source_language='en', target_language = 'ca')))['translatedText']
                                    if textos_items_splitted[i][j][k + 1][0] == traduccion or textos_items_splitted[i][j][k - 1][0] == traduccion:
                                        new_item = (item[0],item_en[1])
                                        textos_items_splitted[i][j][k] = new_item
                                    continue
                            if 0 < k_en and 0 < k < l-1:
                                if textos_items_en_splitted[i][j][k_en - 1][1] != 'O':
                                    traduccion = json.loads(json.dumps(translator.translate(textos_items_en_splitted[i][j][k_en - 1][0],source_language='en', target_language = 'ca')))['translatedText']
                                    if textos_items_splitted[i][j][k + 1][0] == traduccion or textos_items_splitted[i][j][k - 1][0] == traduccion:
                                        new_item = (item[0],item_en[1])
                                        textos_items_splitted[i][j][k] = new_item
                                    continue
                            new_item = (item[0],item_en[1])
                            textos_items_splitted[i][j][k] = new_item

## Visualización Tags Catalán

In [110]:
from IPython.core.display import display, HTML
html = ""
for paragrafo in textos_items_splitted[0]:
    html = html + '<p'
    for item in paragrafo:
            html = html + '<span style = "background-color: ' + tag_to_background_color_en(item[1]) + ';">' + item[0] + ' </span>'
    html = html + '</p>'
display(HTML(html))

## Guardar Dataset Catalan Taggeado

In [111]:
f_dataSet = open(ruta + "DataSet.noTocar", "a", encoding ='utf-8')
f_info = open(ruta + "Info.noTocar", "a", encoding ='utf-8')
for i in range(0,len(matriz_longitudes)):
    f_info.write(lista_paginas[i] + "\n")
    with open(ruta_etiquetados + lista_paginas[i] + ".txt", "w", encoding ='utf-8') as f:
        for paragrafo in textos_items_splitted[i]:
            f.write("\n")
            f_dataSet.write("\n")
            for item in paragrafo:               
                if item[0] != " " :
                    f.write("\n" + item[0] + "\t" + item[1])
                    f_dataSet.write( "\n" + item[0] + "\t" + item[1])
    f.close()
f_info.close()
f_dataSet.close()

# Entrenar NER Catalán

In [112]:
# En Java\Entrenar Clasificador Catalan
# java -cp stanford-ner.jar edu.stanford.nlp.ie.crf.CRFClassifier -prop prop.txt